# 3-4 Feature モジュールの解説と実装

## Feature モジュールのサブネットワークの構成
Feature モジュールの構成を以下に示す．

<img src="../image/p149.png">

1つの FeatureMap_convolution と，それぞれ2つの residualBlockPSP，dilated 版 residualBlockPSP から構成される．
4つ目のサブネットワークから出力されるテンソルは，AuxLoss モジュールに渡される．
学習時には Decoder と AuxLoss 両方の損失値を用いて，より効率的にパラメータを学習させることが出来る．

## サブネットワーク FeatureMap_convolution
FeatureMap_Convolution の詳細なネットワークを次に示す．

<img src="../image/p150.png">

入力画像は事前処理済みの 3 x 475 x 475，出力は 128 x 119 x119 である．
ここでは，単純に畳み込み，バッチノーマライゼーション，マックスプーリングで画像の特徴量を抽出している．

## FeatureMap_convolution の実装
畳み込み層，バッチノーマライゼーション，ReLU をセットにした conv2dBatchNormRelu クラスを作成する．
nn.ReLU(inplace=True) としているが，メモリ効率を重視して ReLU への入力をメモリに保存せず，そのまま出力を計算させている．

In [3]:
import torch
from torch import nn

class conv2dDBatchNormRelu(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dilation, bias):
        super(conv2DBatchNormRelu, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, dilation, bias=bias)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        
        
    def forward(self, x):
        x = self.conv(x)
        x = self.batchnorm(x)
        outputs = self.relu(x)
        
        return outputs
    

class FeatureMap_convolution(nn.Module):
    def __init__(self):
        super(FeatureMap_convolution, self).__init__()
        
        # 畳み込み層1
        in_channels, out_channels, kernel_size, stride, padding, dilation, bias = 3, 64, 3, 2, 1, 1, False
        self.cbnr_1 = conv2DBatchNormRelu(in_channels, out_channels, kernel_size, stride, padding, dilation, bias)
        # 畳み込み層2
        in_channels, out_channels, kernel_size, stride, padding, dilation, bias = 64, 64, 3, 1, 1, 1, False
        self.cbnr_2 = conv2DBatchNormRelu(in_channels, out_channels, kernel_size, stride, padding, dilation, bias)
        # 畳み込み層3
        in_channels, out_channels, kernel_size, stride, padding, dilation, bias = 64, 128, 3, 1, 1, 1, False
        self.cbnr_3 = conv2DBatchNormRelu(in_channels, out_channels, kernel_size, stride, padding, dilation, bias)
        
        # Max Pooling 層
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        
    def forward(self, x):
        x = self.cbnr_1(x)
        x = self.cbnr_2(x)
        x = self.cbnr_3(x)
        outputs = self.maxpool(x)
        
        return outputs

## ResidualBlockPSP
このモジュールでは ResNet の Residual Block 構造を用いる．
以下に ResidualBlockPSP の構造を示す．

<img src="../image/p152.png">

最初に bottleNeckPSP を経て bottleNeckIdentifyPSP を数回繰り返して最終的な出力を計算する．
4回繰り返す ResidualBlockPSP では，それぞれ3，4，6，3回ずつ bottleNeckIdentifyPSP を繰り返す．
この繰り返し回数は任意に設定できるが，ここでは ResNet50 に合わせている．
実装では nn.Module ではなく nn.Sequential を継承している．
nn.Sequential を継承することで forward 関数が予め実装されているため，ResidealBlockPSP の forward は明示的に定義しなくても良い．

In [4]:
class ResidualBlockPSP(nn.Sequential):
    def __init__(self, n_blocks, in_channels, mid_channels, out_channels, stride, dilation):
        super(ResidualBlockPSP, self).__init__()
        
        # bottleNeckPSP の用意
        self.add_module("block1", bottleNeckPSP(in_channels, mid_channels, out_channels, stride, dilation))
        # bottleNeckIdentifyPSP の繰り返しの用意
        for i in range(n_blocks - 1):
            self.add_module("block" + str(i + 2), bottleNeckIdentifyPSP(out_channels, mid_channels, stride, dilation))

## bottleNeckPSP と bottleNeckIdentifyPSP
次に bottleNeckPSP と bottleNeckIdentifyPSP の構造を示す．

<img src="../image/p154.png">

これらのネットワークでは入力が二股に別れて処理される．
下側のルートをスキップ結合，またはショートカットコネクションやバイパスなどと呼ばれる．
2種類のブロックの違いはスキップ結合に畳み込み層が入るか入らないかである．  
スキップ結合は”劣化問題（degradation）”を緩和する役割がある．  
深いネットワークでは浅いネットワークよりも訓練誤差が大きくなることがある．
ブロックへの入力を $x$，上側のルートの出力を $F(x)$，ブロック全体の出力を $y$ とすると，$y = x + F(x)$ となる．
もし，$F(x)$ が 0 となるとブロックの出力は，そのまま $x$ となる．
すると，学習パラメータが全て 0 となっても前段の出力を後段にそのまま伝えることが出来るため，深いネットワークでも劣化問題を避けられる．
すなわち，このブロックでは入力 $x$ をそのまま出力し，望ましい出力 $y$ との残差（residual）$y - x = F(x)$ を学ばせている．   
また，各ブロックには dilation というパラメータが設定されている．
通常の畳み込み層では dilation が1となっている．
dilation が1でない値を使用する畳み込み層を Dilated Convolution と呼ぶ．
これは，下図のように一定の間隔を空けて畳み込みを適用する手法である．
この手法を適用することでより大局的な特徴量を抽出できる．

<img src="../image/p155.png">

In [6]:
class conv2DBatchNorm(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, padding, dilation, bias):
        super(conv2DBatchNorm, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, dilation ,bias=bias)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        
    def forward(self, x):
        x = self.conv(x)
        outputs = self.batchnorm(x)
        return outputs
    

class bottleNeckPSP(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels, kernel_size, stride, dilation):
        super(bottleNeckPSP, self).__init__()
        self.cbr_1 = conv2dBatchNormRelu(in_channels, mid_channels, kernel_size=1, 
                                         stride=1, padding=0, dilation=1, bias=False)
        self.cbr_2 = conv2dBatchNormRelu(mid_channels, mid_channels, kernel_size=3, 
                                         stride=stride, padding=dilation, dilation=dilation, bias=False)
        self.cb_3 = conv2DBatchNorm(mid_channels, out_channels, kernel_size=1,
                                    stride=1, padding=0, dilation=1, bias=False)
        self.cb_residual = conv2DBatchNorm(in_channels, out_channels, kernel_size=1,
                                           stride=stride, padding=0, dilation=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        conv = self.cb_3(self.cbr_2(self.cbr_1(x)))
        residual = self.cb_residual(x)
        return self.relu(conv + residual)
    

class bottleNeckIdentifyPSP(nn.Module):
    def __init__(self, in_channels, mid_channels, stride, dilation):
        super(bottleNeckIdentifyPSP, self).__init__()
        self.cbr_1 = conv2dBatchNormRelu(in_channels, mid_channels, kernel_size=1, 
                                         stride=1, padding=0, dilation=1, bias=False)
        self.cbr_2 = conv2dBatchNormRelu(mid_channels, mid_channels, kernel_size=3, 
                                         stride=1, padding=dilation, dilation=dilation, bias=False)
        self.cb_3 = conv2DBatchNorm(mid_channels, in_channels, kernel_size=1,
                                    stride=1, padding=0, dilation=1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        conv = self.cb_3(self.cbr_2(self.cbr_1(x)))
        residual = x
        return self.relu(conv + residual)